# Import

In [ ]:
import pandas as pd
import os
import spacy
nlp = spacy.load('en_core_web_md')

In [ ]:
import neuralcoref
neuralcoref.add_to_pipe(nlp)

In [ ]:
from ipynb.fs.full.helper_functions import does_contain,get_centralities,get_match,get_sent,sort_dict

# Initialize

In [ ]:
cat = 'National'

In [ ]:
dd = pd.read_csv(f'../data/{cat}/factual.csv')
ndtv = pd.read_csv(f'../data/{cat}/ndtv.csv')
tn = pd.read_csv(f'../data/{cat}/timesNow.csv')

# GetDF

In [ ]:
file_name = 'Centrality Results/data.csv'

In [ ]:
from tqdm import tqdm_notebook
import time

In [ ]:
start_count = 0
if os.path.exists(file_name):
    a = list(pd.read_csv(file_name)['ID'])
    a.reverse()
    start_count = a[0] + 1
end_count = 251
for id in tqdm_notebook(range(start_count,end_count)):
    try:
        ddtext = list(dd[dd['id'] == id].text.items())[0][1]
        ndtvtext = list(ndtv[ndtv['id'] == id].text.items())[0][1]
        tntext = list(tn[tn['id'] == id].text.items())[0][1]

        ddtext = nlp(ddtext)._.coref_resolved
        ndtvtext = nlp(ndtvtext)._.coref_resolved
        tntext = nlp(tntext)._.coref_resolved

        time.sleep(1)
        dd_cent, dd_sent,dd_emotion = get_centralities(ddtext)
        time.sleep(1)
        ndtv_cent, ndtv_sent,ndtv_emotion = get_centralities(ndtvtext)
        time.sleep(1)
        tn_cent, tn_sent,tn_emotion = get_centralities(tntext)
    except:
        print(id)
        continue
    

    dd_cent = sort_dict(dd_cent)
    ndtv_cent = sort_dict(ndtv_cent)
    tn_cent = sort_dict(tn_cent)

    for i in range(len(dd_cent)):
        dd_cent[i] = list(dd_cent[i])
    for i in range(len(ndtv_cent)):
        ndtv_cent[i] = list(ndtv_cent[i])
    for i in range(len(tn_cent)):
        tn_cent[i] = list(tn_cent[i])

    shift_list = []
    dd_tn = []
    dd_ndtv = []
    for dd_ent_rel in dd_cent:
        dd_ent = dd_ent_rel[0].split(" ")
        tn_ent_rel = get_match(dd_ent,tn_cent)
        ndtv_ent_rel = get_match(dd_ent,ndtv_cent)
        if(tn_ent_rel[0] is None and ndtv_ent_rel[0] is None):
            dd_ndtv.append(dd_ent_rel)
            dd_tn.append(dd_ent_rel)
        elif(tn_ent_rel[0] is None):
            dd_tn.append(dd_ent_rel)
        elif(ndtv_ent_rel[0] is None):
            dd_ndtv.append(dd_ent_rel)
        else:
            shift_list.append([dd_ent_rel,ndtv_ent_rel,tn_ent_rel])

    ndtv_dd = []
    ndtv_tn = []
    for ndtv_ent_rel in ndtv_cent:
        ndtv_ent = ndtv_ent_rel[0].split(" ")
        tn_ent_rel = get_match(ndtv_ent,tn_cent)
        dd_ent_rel = get_match(ndtv_ent,dd_cent)
        if(tn_ent_rel[0] is None and dd_ent_rel[0] is None):
            ndtv_dd.append(ndtv_ent_rel)
            ndtv_tn.append(ndtv_ent_rel)
        elif(tn_ent_rel[0] is None):
            ndtv_tn.append(ndtv_ent_rel)
        elif(dd_ent_rel[0] is None):
            ndtv_dd.append(ndtv_ent_rel)   

    tn_ndtv = []
    tn_dd = []
    for tn_ent_rel in tn_cent:
        tn_ent = tn_ent_rel[0].split(" ")
        dd_ent_rel = get_match(tn_ent,dd_cent)
        ndtv_ent_rel = get_match(tn_ent,ndtv_cent)
        if(dd_ent_rel[0] is None and ndtv_ent_rel[0] is None):
            tn_ndtv.append(tn_ent_rel)
            tn_dd.append(tn_ent_rel)
        elif(dd_ent_rel[0] is None):
            tn_dd.append(tn_ent_rel)
        elif(ndtv_ent_rel[0] is None):
            tn_ndtv.append(tn_ent_rel)


    for i in range(len(shift_list)):
        for j in range(i + 1,len(shift_list)):
            if(does_contain(shift_list[i][0][0],shift_list[j][0][0])):
                shift_list[i][0][1] += shift_list[j][0][1]
                shift_list[i][1][1] += shift_list[j][1][1]
                shift_list[i][2][1] += shift_list[j][2][1]
                shift_list[j][0][1] = 0
                shift_list[j][1][1] = 0
                shift_list[j][2][1] = 0
    shift_list = [x for x in shift_list if x[0][1] != 0]

    common_cent = {}
    for tup in shift_list:
        common_cent[tup[0][0]] = [tup[0][1],tup[1][1],tup[2][1]]

    common_sent = {}
    for key in common_cent:
        common_sent[key] = [get_sent(key,dd_sent), get_sent(key,ndtv_sent), get_sent(key,tn_sent)]
    
    common_emo = {}
    for key in common_cent:
        common_emo[key] = [get_sent(key,dd_emotion),get_sent(key,ndtv_emotion),get_sent(key,tn_emotion)]

    df = pd.DataFrame(columns=['ID','dd_text','ndtv_text','tn_text','dd_centralities','ndtv_centralities','tn_centralities','common_centralities','common_sentiment','common_emotion','dd_tn','dd_ndtv','tn_dd','tn_ndtv','ndtv_dd','ndtv_tn','dd_emotion','ndtv_emotion','tn_emotion'])
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    row =  {'ID': id,'dd_text': ddtext, 'ndtv_text':ndtvtext, 'tn_text' : tntext, 'common_centralities' : common_cent,'common_sentiment' : common_sent,
            'dd_tn' : dd_tn, 'dd_ndtv' : dd_ndtv,'tn_dd' : tn_dd,'tn_ndtv' : tn_ndtv,
            'ndtv_dd' : ndtv_dd,'ndtv_tn' : ndtv_tn,'dd_centralities' : dd_cent,'ndtv_centralities' : ndtv_cent,
            'tn_centralities': tn_cent,'common_emotion' : common_emo,'dd_emotion' : dd_emotion,'ndtv_emotion': ndtv_emotion,'tn_emotion':tn_emotion}
    df.loc[len(df)] = row
    df.to_csv(file_name,index = False)